In [1]:
include("../code/julia/ha-trade.jl")
include("../code/julia/ha-trade-welfare.jl")

using MINPACK
using Plots
using CSV
using DataFrames

### First step, match up on no quality version...

In [2]:
γ = 1.5 # curvatuve on CRRA utility function
σϵ = 0.25 # logit dispersion parameter
Ncntry = 2 # number of countries

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ, ψslope = 0.0);

#ψslope controls how residual demand varies with income state...

In [3]:
# here are some simpe country parametrers
TFP = [1.0; 1.0]

τ = [0.0; 0.0]

L = [1.0; 1.0]

d_ij = 1.745

d = [1.0 d_ij; d_ij 1.0]

# this sets up the country specific paramters
cntry_prm = country_params(Ncntry = Ncntry, L = L, d = d, TFP = TFP);

In [4]:
f(x) = world_equillibrium_FG(exp.(x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the finacial globalization case

function f!(fvec, x)

    fvec .= f(x)

end


xguess = [1.0; 1.02]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, log.(xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

# This solver works very well in general. Spencer Lyon converted old-school minpack
# routines in C into julia

print(sol)

wage = [exp.(sol.x[1]); 1.0]
R = exp.(sol.x[2])

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.560862e-01     0.000000e+00        42.693000
     2     1.456059e-02     2.654986e-05         1.079000
     3     1.638137e-03     4.506004e-07         0.174000
     4     1.812301e-05     7.164351e-09         0.139000
     5     1.674584e-08     9.008308e-13         0.192000
     6     6.698718e-11     8.321748e-19         0.194000
     7     3.676642e-13     1.185695e-23         0.167000
     8     1.179612e-15     3.365318e-28         0.140000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [0.0, 0.01980262729617973]
 * Zero: [-2.039586404137559e-9, 0.014094023815588037]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 44.790000 seconds
 * Function Calls: 8
 * Jacobian Calls (df/dx): 1

1.0141938128276817

In [5]:
Rsol = [R; R]

Y, tradeflows, A_demand, Gbudget, tradeshare, hh, dist = world_equillibrium(Rsol, wage, τ, hh_prm, cntry_prm, tol_vfi = 1e-10);

In [6]:
R

1.0141938128276817

In [7]:
tradeshare

2×2 Matrix{Float64}:
 0.900484   0.0995163
 0.0995163  0.900484

In [8]:
hh[1].πprob[:,:,2]
# guys in country one, a X z, probability of buying from 2

100×10 Matrix{Float64}:
 0.00747609  0.0192432  0.0202433  0.0386541  …  0.103814  0.123432  0.138619
 0.0139284   0.0212163  0.0281047  0.0428235     0.105277  0.127093  0.139288
 0.0194756   0.0264393  0.0357197  0.0456883     0.106613  0.129608  0.139941
 0.0218411   0.0282292  0.0387705  0.0478642     0.107806  0.131053  0.140583
 0.0265513   0.0302667  0.042976   0.0506709     0.108911  0.132266  0.141216
 0.0285041   0.0333028  0.0457674  0.0530594  …  0.109967  0.133278  0.141837
 0.030625    0.0352261  0.0480061  0.0548275     0.111015  0.134253  0.142449
 0.0333905   0.0371223  0.0507749  0.0564595     0.112049  0.135159  0.143062
 0.0354688   0.039016   0.0531147  0.0582905     0.112984  0.136008  0.143643
 0.037335    0.0409391  0.0548801  0.0601975     0.113907  0.136789  0.144216
 0.0392264   0.0426161  0.0565561  0.061869   …  0.114807  0.137517  0.14478
 0.0411777   0.0442844  0.058384   0.0633612     0.115682  0.138219  0.145336
 0.0428169   0.0459213  0.0602438  0.0647

### Now turn quality on...

In [9]:

# this setups up parameters on the household side
hh_prm = household_params(Ncntry = Ncntry, Na = 100, β = 0.92, γ = γ, ϕ = 0.5, amax = 8.0, σϵ = σϵ, ψslope = 0.2);

#ψslope controls how residual demand varies with income state...

In [10]:
f(x) = world_equillibrium_FG(exp.(x), hh_prm, cntry_prm)
# this world... function is used to construct zero conditions for
# the finacial globalization case

function f!(fvec, x)

    fvec .= f(x)

end


xguess = [1.0; 1.02]

n = length(xguess)
diag_adjust = n - 1

sol = fsolve(f!, log.(xguess), show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-10,
       )

# This solver works very well in general. Spencer Lyon converted old-school minpack
# routines in C into julia

print(sol)

wage = [exp.(sol.x[1]); 1.0]
R = exp.(sol.x[2])

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.889034e-01     0.000000e+00         0.155000
     2     2.011610e-02     3.677478e-05         0.444000
     3     2.740911e-03     9.362682e-07         0.120000
     4     4.368312e-05     2.306130e-08         0.173000
     5     2.926494e-07     5.603311e-12         0.141000
     6     7.476779e-10     2.653056e-16         0.136000
     7     1.251874e-12     1.677989e-21         0.158000
     8     7.216450e-16     4.646174e-27         0.219000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [0.0, 0.01980262729617973]
 * Zero: [-2.3167443794377336e-9, 0.01299023862956576]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 1.546000 seconds
 * Function Calls: 8
 * Jacobian Calls (df/dx): 1

1.013074978311403

In [11]:
Rsol = [R; R]

Y, tradeflows, A_demand, Gbudget, tradeshare, hh, dist = world_equillibrium(Rsol, wage, τ, hh_prm, cntry_prm, tol_vfi = 1e-10);

In [12]:
R

1.013074978311403

In [13]:
tradeshare

2×2 Matrix{Float64}:
 0.919293   0.0807066
 0.0807066  0.919293

In [14]:
hh[1].πprob[:,:,2]
# guys in country one, a X z, probability of buying from 2

100×10 Matrix{Float64}:
 0.0153874  0.0389713  0.0289009  …  0.0749608  0.0640053  0.0728029
 0.0284216  0.042925   0.0399521     0.0760607  0.0661423  0.0731829
 0.0394186  0.0529172  0.05056       0.0770306  0.0676554  0.0735568
 0.0441193  0.056503   0.0547915     0.0779161  0.0685637  0.0739241
 0.0531311  0.0605085  0.0605846     0.0787469  0.0692215  0.0742847
 0.0570407  0.0661851  0.0644315  …  0.0795477  0.069815   0.0746472
 0.0611752  0.0699163  0.0675571     0.0803421  0.0703682  0.0749934
 0.0663712  0.073557   0.0713012     0.0810946  0.0708869  0.0753281
 0.0703746  0.0771641  0.0744958     0.0818012  0.0713386  0.0756581
 0.0739593  0.0807869  0.0769521     0.0824948  0.0717678  0.0759826
 0.0775502  0.0839139  0.0792597  …  0.0831723  0.0721788  0.0763022
 0.081191   0.0870507  0.0817287     0.083828   0.0725769  0.0766183
 0.0843079  0.0901137  0.0842488     0.0844674  0.0729753  0.0769312
 ⋮                                ⋱                        
 0.206205   0.20802